<a href="https://colab.research.google.com/github/luizvalle/Chessformer/blob/main/chessformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
# @title Install dependencies {display-mode: "form"}
%%shell
pip install chess
pip install zstandard
pip install pandas
pip install numpy
pip install dask

In [52]:
# @title Imports {display-mode: "form"}
import chess.pgn
import zstandard as zstd
import requests
import io
import pandas as pd
import numpy as np
import dask.dataframe as dd
import re

from pprint import pprint

## Download and prepare the data

In [17]:
# @title Fetch the dowload links {display-mode: "form"}
DOWNLOAD_LIST = "https://database.lichess.org/standard/list.txt"
download_links = requests.get(DOWNLOAD_LIST).text.split('\n')
print("Examples:")
pprint(download_links[0:5])

Examples:
['https://database.lichess.org/standard/lichess_db_standard_rated_2023-09.pgn.zst',
 'https://database.lichess.org/standard/lichess_db_standard_rated_2023-08.pgn.zst',
 'https://database.lichess.org/standard/lichess_db_standard_rated_2023-07.pgn.zst',
 'https://database.lichess.org/standard/lichess_db_standard_rated_2023-06.pgn.zst',
 'https://database.lichess.org/standard/lichess_db_standard_rated_2023-05.pgn.zst']


In [18]:
# @title Define the class to iterate through games {display-mode: "form"}
class CompressedPgnIterator:
  def __init__(self, download_link):
      dctx = zstd.ZstdDecompressor()
      # Stream the results so we do not load everything
      # into memory at once
      response = requests.get(url=download_link, stream=True)
      reader = dctx.stream_reader(response.raw)
      self.text_stream = io.TextIOWrapper(reader, encoding='utf-8')

  def __iter__(self):
      return self

  def __next__(self):
    game = chess.pgn.read_game(self.text_stream)
    if game:
      return game
    else:
      raise StopIteration

In [57]:
column_types = {
    "Event": "category",
    "Result": "category",
    "WhiteElo": "int16",
    "BlackElo": "int16",
    "TimeControl": "category",
    "Termination": "category"
}
for download_link in download_links:
  date = re.search(r"(\d{4}-\d{2}).pgn.zst", download_link).group(1)
  games = CompressedPgnIterator(download_link)
  games_info = list()
  for game in games:
    game_info = {
        "Event": game.headers["Event"],
        "Result": game.headers["Result"],
        "WhiteElo": int(game.headers["WhiteElo"]),
        "BlackElo": int(game.headers["BlackElo"]),
        "TimeControl": game.headers["TimeControl"],
        "Termination": game.headers["Termination"]
    }
    games_info.append(game_info)
    break
  df = pd.DataFrame(data=games_info, dtype=column_types).astype(column_types)
  df.to_parquet(path="")


https://database.lichess.org/standard/lichess_db_standard_rated_2023-09.pgn.zst 2023-09


ValueError: ignored